In [149]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
import os

In [150]:
DATA_IN_PATH = '../../source/data/'
DATA_OUT_PATH = '../data_out/'

In [176]:
#load in file high
df = pd.read_csv(DATA_IN_PATH + 'high.csv')

#get coin list
coins = [i for i in df.columns]
coins.remove('Date')

#add prefix high
df = df.add_prefix('high ')
#remove prefix from Date
df.rename(columns={'high Date' : 'Date'}, inplace=True)

#remaining files list
files = ['adj_close', 'close', 'low', 'open', 'volume']
#for each file
for file_ in files:
    #load file & add prefix
    file_df = pd.read_csv(DATA_IN_PATH + file_ + '.csv').add_prefix(file_ + ' ')
    #remove prefix from Date
    file_df.rename(columns={file_ + ' Date' : 'Date'}, inplace=True)
    #merge on Date
    df = pd.merge(df, file_df, on='Date')
    
#set date as dt
df.Date = pd.to_datetime(df.Date)
#set date as index
df.set_index('Date', inplace=True)
#set dt freq to day
df = df.asfreq('d')
df = df.sort_index()

In [177]:
df.head(3)

,high ADA-USD,high ATOM-USD,high AVAX-USD,high AXS-USD,high BTC-USD,high ETH-USD,high LINK-USD,high LUNA1-USD,high MATIC-USD,high SOL-USD,...,volume ADA-USD,volume ATOM-USD,volume AVAX-USD,volume AXS-USD,volume BTC-USD,volume ETH-USD,volume LINK-USD,volume LUNA1-USD,volume MATIC-USD,volume SOL-USD
Date,,,,,,,,,,,,,,,,,,,,,
2017-04-26,NaN,NaN,NaN,NaN,1294.829956,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,329631008,NaN,NaN,NaN,NaN,NaN
2017-04-27,NaN,NaN,NaN,NaN,1319.699951,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,449196992,NaN,NaN,NaN,NaN,NaN
2017-04-28,NaN,NaN,NaN,NaN,1331.280029,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,527488992,NaN,NaN,NaN,NaN,NaN


In [180]:
df.tail(3)

,high ADA-USD,high ATOM-USD,high AVAX-USD,high AXS-USD,high BTC-USD,high ETH-USD,high LINK-USD,high LUNA1-USD,high MATIC-USD,high SOL-USD,...,volume ADA-USD,volume ATOM-USD,volume AVAX-USD,volume AXS-USD,volume BTC-USD,volume ETH-USD,volume LINK-USD,volume LUNA1-USD,volume MATIC-USD,volume SOL-USD
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-23,0.910614,23.452826,74.369293,46.072273,39935.859375,2975.322754,13.643617,93.895706,1.406944,102.771233,...,4.479977e+08,262237459.0,360787956.0,185825882.0,16138021249,9.116956e+09,322384215.0,1.251243e+09,549446712.0,8.232525e+08
2022-04-24,0.897855,23.259838,73.437004,45.520817,39845.925781,2961.882080,13.608301,92.183968,1.373304,102.216682,...,4.876295e+08,296530518.0,320340266.0,171259555.0,17964398167,9.696830e+09,299368814.0,1.256729e+09,420960265.0,8.182660e+08
2022-04-25,0.901707,22.330229,72.904549,44.428181,40491.753906,3018.415527,13.487024,97.251686,1.354482,101.440590,...,1.139869e+09,515036727.0,735322462.0,342114485.0,35445730570,2.233269e+10,506454272.0,2.826796e+09,713007261.0,1.538487e+09


In [178]:
df.shape

(1826, 60)

In [179]:
coins

['ADA-USD',
 'ATOM-USD',
 'AVAX-USD',
 'AXS-USD',
 'BTC-USD',
 'ETH-USD',
 'LINK-USD',
 'LUNA1-USD',
 'MATIC-USD',
 'SOL-USD']

### Split (test model)

In [154]:
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [138]:
#timesplit
#https://github.com/scikit-learn/scikit-learn/issues/22523
#x = np.arange(15)
#cv = TimeSeriesSplit(max_train_size=3, test_size=1)
#for train_index, test_index in cv.split(x):
#    print("TRAIN:", train_index, "TEST:", test_index)

#scale each x_train


#https://www.cienciadedatos.net/documentos/py27-time-series-forecasting-python-scikitlearn.html
#https://www.tensorflow.org/tutorials/structured_data/time_series

In [159]:
days = 30

#coin = 'BTC-USD'

#df_ = df[f'{coin}']

df_ = df.copy()

X, y = [], []

tscv = TimeSeriesSplit(max_train_size=days, test_size=1, n_splits=df_.shape[0]-days)

for train_index, test_index in tscv.split(df_):
    X.append(df_.iloc[train_index].values)
    y.append(df_.iloc[test_index])
    
    
#X_train, y_train = np.array(X_train), np.array(y_train)
    
    #print(y)
    
    
    #model = Sequential()
    #model.add(LSTM(units=30, return_sequences=True, input_shape=(X.shape[0], 1)))
    #model.compile(optimizer = 'adam', loss='mean_absolute_error')
    #model.fit(X, y)
    #result = model.predict(y)
    #results.append(result)

In [181]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [182]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

#### Scale